# Create variables

In [1]:
import os, sys
sys.path.append(os.path.abspath('../../'))
import tally
dataset = tally.DataSet(api_key=os.environ.get('tally_api_key'))
dataset.use_spss('./data/Example Data (A).sav')

## derive - Creating new variables

Let's say we've run a survey asking people about what sports they do. We want to create a new variable that designates people who do sports/excersize regularly and have their main fitness activity as aerobics, yoga or pilates. 

### Exploring the relevant codes

In order to do this we look at the meta data for questions `q1` and `q2b` and find the codes for aerobics, yoga, pilates and regular sporties. These are 4, 5, 6 from `q1` and 1 for `q2b`.

In [2]:
# what is your main sports activity?
dataset.meta(variable='q1')

,codes,texts,missing
1,1,Swimming,None
2,2,Running/jogging,None
3,3,Lifting weights,None
4,4,Aerobics,None
5,5,Yoga,None
6,6,Pilates,None
7,7,Football (soccer),None
8,8,Basketball,None
9,9,Hockey,None
10,96,Other,None


In [3]:
# how regularly do you excersise
dataset.meta(variable='q2b')

,codes,texts,missing
1,1,Regularly,None
2,2,Irregularly,None
3,3,Never,None


### Building the logic

We now build the logic for our new variable. Codes 1 will represent "regular sporties, mainly into yoga, aerobics or pilates" and code 2 will include non sporties and those who don't have the above sports as their main sports. We use the Tally logical system as [documented in the API.](https://tally.datasmoothie.com/#section/Selecting-responses-with-logic-operators)

In [4]:
cond_map = [
    (
        1, 
        "Regular sporties, mainly into yoga, aerobics or pilates", 
        { "$intersection": [{"q1":[4, 5, 6]}, {"q2b":[1]}] }
    ),
    (
        2, 
        "Non sporties, main activity not yoga, airobics, pilates", 
        {"$union":
            [
                {"$not_any":{"q2b":[1]}},
                {"$not_any":{"q1":[4,5,6]}}
            ]
        }
    )
]

### Creating the derived variable

Using our new logic, we create the variable with the derive method (for more details, [see the API on derive](https://tally.datasmoothie.com/#tag/Data-Processing/operation/derive)).

In [5]:
dataset.derive(
    name='active_aer_yoga_pilates', 
    label='Active sporties into aerobics, yoga or pilates', 
    cond_maps=cond_map, 
    qtype="single"
)

Once we have created the new variable, we can sanity check it.

In [6]:
dataset.crosstab(x='active_aer_yoga_pilates', ci=['counts', 'c%'])

Question                                                                                            Total
Values                                                                                              Total
Question                                       Values                                                    
Active sporties into aerobics, yoga or pilates Base                                                8255.0
                                               Regular sporties, mainly into yoga, aerobics or...   339.0
                                                                                                      4.1
                                               Non sporties, main activity not yoga, airobics,...  7916.0
                                                                                                     95.9